# Regresión Lineal

## Introducción

Un modelo de regresión lineal predice la respuesta como la suma ponderada de features. La linealidad de la relación aprendida hace que la interpretación sea simple.

Ls modelos lineales pueden utilizarse para modelar la dependencia de una variable respuesta **y** versus un conjunto de features **x**. La relación aprendida puede escribirse como:

$$ y = \beta_0 + \beta_1 X_1 +\beta_2 X_2 + ... + \beta_p X_p + \epsilon $$

El valor predicho para una instancia es la suma ponderada de sus p features.  Los $\beta_i$ representan los pesos de los features. El primero es $\beta_0$ que se conoce como intercepto y no se multiplica por ningún feature, por otro lado $\epsilon$ es el error que aún se comete, esto es, la diferencia entre la perdicción y el verdadero valor. Estos errores se asumen con una distribución normal.

Para encontrar el mejor coeficiente, se suele minimizar el cuadrado de la diferencia entre los valores estimados y los valores actuales. 
La principal ventaja de los modelos de regresión es la linealidad, hace que el proceso de estimación sea simple y lo más importante es que estas ecuaciones tienen una interpretación muy fácil de entender.

Los pesos estimados se acompañan con intervalos de confianza, en el cual se estima que el peso cubre al verdadero valor con una cierta confianza. 

Si el modelo es *"correcto"* depende si las relaciones en los datos cumplen ciertos supuestos que son:

- **Linealidad**: los modelos de regresión son una combinación lineal de variables, lo cual es una gran fortaleza y una grande limitación. La linealidad resulta en modelos interpretables que son fáciles de cuantificar y describir. Son aditivos, lo que implica que es posible aislar efectos.  Si se sospecha interacción de variables o asociación no lineal, pueden incluirse términos de interacción o splines.

- **Normalidad**: se asume que la variable respuesta sigue una distribución normal, si este supuesto no se cumple, los intervalos de confianza estimados son inválidos.

- **Homocedasticidad**:  se asume que la variancia del error es constante en el tiempo. Este es un supuesto que no se suele cumplir en la realidad.

- **Independencia**: se asume que cada instancia es independiente de otra instancia. 

- **Efectos Fijos**: los features son considerados fijos, esto quiere decir que son tratados como constantes y no como variables aleatorias. Esto implica que son libres de erroes de medición, lo cual es otro supuesto poco realista. Sin embargo, sin este supuesto, debería ajustarse modelos mucho más complejos.

- **Ausencia de multicolinealidad**: no se desea que haya variables altamente correlacionadas porque esto arruina los pesos de los features, hace que sea muy difícil estimar los pesos. 

## Interpretación

La interpretación del modelo de regresión va a depender del tipo de variable:

- **Feature numérico**: ante cambios unitarios de la variable, aumenta la respuesta en el peso estimado para dicha variable.

- **Feature binario** : es una variable que toma dos posibles valores (0: ausencia del atributo - 1: presencia del atributo). El coeficiente indica cuánto cambia la respuesta al pasar del valor de referencia (0) al otro valor (1).

- **Variable con N categorías**: una variable con un número fijo de valores posibles (A-B-C-D). Una solución es usar *codificación one-hot*, que implica que cada categoría tiene su propia columna binaria. Para variables categóricas con D categorías, se necesitan D-1 columnas. La interpretación para cada categoría es similar al caso binario, se mide el cambio respecto a la variable de referencia.

- **Intercepto**: el intercepto es un feature constante, como si fuera que la variable que acompaña a ese coeficiente vale 1. Se interpreta como, cuando todos los feature valen cero y todas las variables categóricas se encuentran en su valor de referencia, la predicción del modelo es el peso del intercepto.

- **Feature numérico**: el aumento del feature $x_j$ en una unidad aumenta/disminuye la predicción de $y$ en $\beta_j$ unidades cuando las demás variables permanecen constantes.

- **Feature categórcio**:  pasar de la categoría de referencia de $x_j$ a otra categoría, aumenta/disminuye la predicción de  $y$ en $\beta_j$ unidades cuando las demás variables permanecen constantes.

Otra métrica importante paara interpretar los modelos de regresión es el $R^2$, mientras más alto mejor explica el modelo los datos. El tema es que siempre aumenta al aumentar el número de features, es por esto que es mejor usar $R^2_{ajustado}$.

### Feature importance

La importancia de una variable en un modelo de regresión lineal puede ser medido como el valor absoluto de la **estadística t**. 

$$t_{\hat{\beta_j}} = \frac{\hat{\beta_j}}{SE(\hat{\beta_j})}

La importancia de una variable aumenta al aumentar su coeficiente, mientras mayor variancia tenga el peso estimado (menos seguridad sobre el valor correcto), menos importante debe ser el feature.

## Aplicación

### Librerías

In [10]:
import os 
import re
import pandas as pd
import numpy as np
import plotly.express as px
import pyreadr
import plotly.graph_objects as go
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

### Lectura de datos

In [2]:
result = pyreadr.read_r('../data/bike.RData')

# Esto devuelve un diccionario donde las claves son los nombres de los objetos R
print(result.keys())

df = result['bike']


df.head()

odict_keys(['bike'])


,instant,dteday,season,yr,mnth,holiday,weekday,temp,atemp,hum,windspeed,casual,registered,cnt,workday,weather,days_since_2011,cnt_2d_bfr
0,3,2011-01-03,WINTER,2011,JAN,N,MON,1.229108,22.439770,43.7273,16.636703,120,1229,1349,Y,GOOD,2.0,985
1,4,2011-01-04,WINTER,2011,JAN,N,TUE,1.400000,23.212148,59.0435,10.739832,108,1454,1562,Y,GOOD,3.0,801
2,5,2011-01-05,WINTER,2011,JAN,N,WED,2.666979,23.795180,43.6957,12.522300,82,1518,1600,Y,GOOD,4.0,1349
3,6,2011-01-06,WINTER,2011,JAN,N,THU,1.604356,23.929106,51.8261,6.000868,88,1518,1606,Y,GOOD,5.0,1562
4,7,2011-01-07,WINTER,2011,JAN,N,FRI,1.236534,23.100526,49.8696,11.304642,148,1362,1510,Y,MISTY,6.0,1600


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   instant          728 non-null    int32   
 1   dteday           728 non-null    object  
 2   season           728 non-null    category
 3   yr               728 non-null    category
 4   mnth             728 non-null    category
 5   holiday          728 non-null    category
 6   weekday          728 non-null    category
 7   temp             728 non-null    float64 
 8   atemp            728 non-null    float64 
 9   hum              728 non-null    float64 
 10  windspeed        728 non-null    float64 
 11  casual           728 non-null    int32   
 12  registered       728 non-null    int32   
 13  cnt              728 non-null    int32   
 14  workday          728 non-null    category
 15  weather          728 non-null    category
 16  days_since_2011  728 non-null    float64 
 1

### Transformación de datos

In [4]:
variables = ['season', 'holiday', 'workday', 'weather', 'temp', 'hum', 'windspeed', 'cnt_2d_bfr']
X = df[variables].copy()
y = df['cnt']


In [5]:
X['weather'] = pd.Categorical(X['weather'], categories=['GOOD', 'MISTY', 'BAD'], ordered=True)
X['season']  = pd.Categorical(X['season'],  categories=['WINTER', 'SPRING', 'SUMMER', 'FALL'], ordered=True)
X['holiday'] = pd.Categorical(X['holiday'], categories=['N', 'Y'], ordered=True)
X['workday'] = pd.Categorical(X['workday'], categories=['N', 'Y'], ordered=True)

In [6]:
X = pd.get_dummies(X, drop_first=True)


In [7]:
X = X.astype(float)


### Ajuste del modelo

***Sin estandarizar***

In [8]:
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     202.1
Date:                Wed, 15 Oct 2025   Prob (F-statistic):          4.52e-211
Time:                        16:49:14   Log-Likelihood:                -6024.6
No. Observations:                 728   AIC:                         1.207e+04
Df Residuals:                     716   BIC:                         1.213e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          2562.7653    267.002      9.598

***Con estandarización***

In [19]:
variables2 = ['season', 'holiday', 'workday', 'weather', 'temp', 'hum', 'windspeed', 'cnt_2d_bfr']
X2 = df[variables].copy()
y2 = df['cnt']

In [20]:
X2['weather'] = pd.Categorical(X2['weather'], categories=['GOOD', 'MISTY', 'BAD'], ordered=True)
X2['season']  = pd.Categorical(X2['season'], categories=['WINTER', 'SPRING', 'SUMMER', 'FALL'], ordered=True)
X2['holiday'] = pd.Categorical(X2['holiday'], categories=['N', 'Y'], ordered=True)
X2['workday'] = pd.Categorical(X2['workday'], categories=['N', 'Y'], ordered=True)


In [21]:
# Crear dummies de variables categóricas
categoricas = ['season', 'holiday', 'workday', 'weather']
X2 = pd.get_dummies(X2, drop_first=True)

In [22]:
numericas = ['temp', 'hum', 'windspeed', 'cnt_2d_bfr']
scaler = StandardScaler()
X2[numericas] = scaler.fit_transform(X2[numericas])

In [23]:
X2 = X2.astype(float)

# Agregar constante
X2 = sm.add_constant(X2)

# Ajustar modelo OLS
model2 = sm.OLS(y2, X2).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.753
Method:                 Least Squares   F-statistic:                     202.1
Date:                Wed, 15 Oct 2025   Prob (F-statistic):          4.52e-211
Time:                        17:06:11   Log-Likelihood:                -6024.6
No. Observations:                 728   AIC:                         1.207e+04
Df Residuals:                     716   BIC:                         1.213e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          4304.8757    120.927     35.599

### Weight plot

Es importante recordar que:
- el peso de un feature depende *de la unidad en el que está medido dicho feature*. 
- si tengo features en diferentes escalas no debo comparar directamente los pesos. Un coeficiente grande puede reflejar que tiene una escala mayor dicho feature.



El **weight plot** grafica los pesos de los features y sus intervalos de confianza. Es útil para ver rápidamente si un coeficiente es distinto de cero y su incertidumbre. 

In [11]:
coef_df = pd.DataFrame({
    'Variable': model.params.index,
    'Coeficiente': model.params.values,
    'CI_lower': model.conf_int()[0],
    'CI_upper': model.conf_int()[1]
})
coef_df = coef_df[coef_df['Variable'] != 'const'] #quitar el intercepto
coef_df['error'] = coef_df['CI_upper'] - coef_df['Coeficiente']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=coef_df['Coeficiente'],
    y=coef_df['Variable'],
    error_x=dict(
        type='data',
        symmetric=False,
        array=coef_df['CI_upper'] - coef_df['Coeficiente'],
        arrayminus=coef_df['Coeficiente'] - coef_df['CI_lower']
    ),
    mode='markers',
    marker=dict(color='blue', size=10),
    orientation='h'
))

fig.update_layout(
    title='Forest Plot - Coeficientes del Modelo',
    xaxis_title='Coeficiente',
    yaxis_title='Variable',
    yaxis=dict(autorange="reversed")  # Para que la primera variable quede arriba
)

fig.show()

In [24]:
coef_df2 = pd.DataFrame({
    'Variable': model2.params.index,
    'Coeficiente': model2.params.values,
    'CI_lower': model2.conf_int()[0],
    'CI_upper': model2.conf_int()[1]
})
coef_df2 = coef_df2[coef_df2['Variable'] != 'const'] #quitar el intercepto
coef_df2['error'] = coef_df2['CI_upper'] - coef_df2['Coeficiente']

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=coef_df2['Coeficiente'],
    y=coef_df2['Variable'],
    error_x=dict(
        type='data',
        symmetric=False,
        array=coef_df2['CI_upper'] - coef_df2['Coeficiente'],
        arrayminus=coef_df2['Coeficiente'] - coef_df2['CI_lower']
    ),
    mode='markers',
    marker=dict(color='blue', size=10),
    orientation='h'
))

fig.update_layout(
    title='Forest Plot - Coeficientes del Modelo',
    xaxis_title='Coeficiente',
    yaxis_title='Variable',
    yaxis=dict(autorange="reversed")  # Para que la primera variable quede arriba
)

fig.show()

### Effect plot

Para una observación i, el efecto (contribución) del feature j a la predicción es:

$$effect_{i,j}= \beta_jx_{ij}$$

La predicción es la suma de todas las contribuciones más el intercepto

$$\hat{y_i}= \beta_0+ \sum_j effect_{i,j}$$

El **effect plot** para cada feature grafica la distribución de los pesos por los valores de los features a lo largo del dataset. Muestra cuánto aporta en la práctica esa variable a las predicciones.

In [26]:
numeric_vars = ['temp', 'hum', 'windspeed', 'cnt_2d_bfr']
categorical_vars = ['season', 'holiday', 'workday', 'weather']
dummies = {var: [col for col in X.columns if col.startswith(var+'_')] for var in categorical_vars}
effects_numeric = X[numeric_vars] * model.params[numeric_vars] 
effects_categorical = pd.DataFrame()
for var, cols in dummies.items():
    effects_categorical[var] = X[cols].dot(model.params[cols])
effects_total = pd.concat([effects_numeric, effects_categorical], axis=1)
effects_long = effects_total.reset_index().melt(id_vars='index', var_name='Feature', value_name='Effect')
fig = px.box(effects_long, y='Feature', x='Effect', 
#points='all',
             title='Effect Plot - Distribución del efecto de cada feature',
             height=800)
fig.show()

In [25]:
numeric_vars2 = ['temp', 'hum', 'windspeed', 'cnt_2d_bfr']
categorical_vars2 = ['season', 'holiday', 'workday', 'weather']
dummies2 = {var: [col for col in X2.columns if col.startswith(var+'_')] for var in categorical_vars2}
effects_numeric2 = X2[numeric_vars2] * model.params[numeric_vars2] 
effects_categorical2 = pd.DataFrame()
for var, cols in dummies2.items():
    effects_categorical2[var] = X2[cols].dot(model2.params[cols])
effects_total2 = pd.concat([effects_numeric2, effects_categorical2], axis=1)
effects_long2 = effects_total2.reset_index().melt(id_vars='index', var_name='Feature', value_name='Effect')
fig2 = px.box(effects_long2, y='Feature', x='Effect', 
#points='all',
             title='Effect Plot - Distribución del efecto de cada feature',
             height=800)
fig2.show()

### Significación 

Weight ($\beta_j$): es el coeficiente de regresión estimado por el modelo.

Effect ($\beta_jx_{i,j}$): es la contribución real de la variable a la predicción de una observación en particular.


Los intervalos de confianza, los t-values, p-values se calculan siempre sobre los **weights**.

**Nunca** se usan los effects para ver si el feature es significativo o no.


¿Qué significa que el intervalo de confianza de los pesos incluya al cero?  Significa que el feature no es significativo.



¿Que significa que la distribución de los efectos incluya al cero? Significa que para algunas instancias el efecto puede ser negativo, positivo o nulo. 